<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Mortgage-scenarios" data-toc-modified-id="Mortgage-scenarios-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Mortgage scenarios</a></span></li><li><span><a href="#Kiwibuild,-new-build" data-toc-modified-id="Kiwibuild,-new-build-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Kiwibuild, new build</a></span></li><li><span><a href="#New-build-in-Auckland" data-toc-modified-id="New-build-in-Auckland-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>New build in Auckland</a></span></li><li><span><a href="#Existing-build-in-Auckland" data-toc-modified-id="Existing-build-in-Auckland-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Existing build in Auckland</a></span></li><li><span><a href="#Out-of-Auckland" data-toc-modified-id="Out-of-Auckland-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Out of Auckland</a></span></li><li><span><a href="#Leasehold" data-toc-modified-id="Leasehold-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Leasehold</a></span></li><li><span><a href="#Commercial" data-toc-modified-id="Commercial-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Commercial</a></span></li></ul></div>

## Mortgage scenarios

Housing is nuts. This is a quick (rough) running of the numbers for a number of the options that I am considering, the end goal (at the moment) being able to live/work in a small to medium sized warehouse in West Auckland somewhere in five years time. There are a small number of suitable warehouses going at the moment in the 6-700K range. The problem with these is that banks want a much greater deposit (~40%) and have much shorter terms for loans on commercial buildings.

I'm not ruling anything out. I lie. I am ruling some things out, a house is not everything. I am however, thinking options through before discarding them. There are a number of scenarios below, all to be evaluated against the end goal. 

For the next 9 months or so, whilst my salary stays under 95K I should theoretically have access to the First Home Grants and Loans. These are not huge - the First Home Grant is 
- 1k for every year of contributions to kiwisaver to a maximum of 5k for an existing build
- 2k for every year of contributions to kiwisaver to a maximum of 10k for an new build
The first home loans enable a mortgage with a 5\% deposit, to a maximum of 625K for an existing build or 700K for a new build. The only proviso is that one has to live in the building for at least six months. 

I haven't got to evaluate all of these yet, but enough of them to get an idea of what is viable given different deposit amounts, and then evaluate against the end goal criteria. They are all by necessity, approximate. Some assumptions used:
- higher than current interest rate, to guard against the fact that the interest rates will go up. 
- capital gains are only assumed on some options
- capital gains are assumed to be lower than they have been for the last few years



In [1]:
# relevant code/imports for all sections. 

from ipywidgets import interact, widgets, Layout
from ipywidgets import VBox, HBox, Label, IntSlider
from IPython.display import display
from mortgage import Loan
import inspect
from IPython.display import Markdown as md
import pandas as pd
from IPython.display import display, HTML
from ipywidgets import AppLayout


# for widget description width
style = {'description_width': 'initial'}

def installmentToDict(installment):
    installmentDict=dict()
    for i in inspect.getmembers(installment):
    # to remove private and protected functions
        if not i[0].startswith('_'):
        # To remove other methods that do not start with a underscore
            if not inspect.ismethod(i[1]): 
                installmentDict[i[0]]=i[1]
    del installmentDict["count"]
    del installmentDict["index"]
    
    #for key, value in installmentDict.items():
    #    if isinstance(value, Decimal):
    #        installmentDict[key] = float(value)

    return installmentDict

def compound_gains(principle, rate, time):
    # time is in years
    n = 1 #number of periods at which the interest is accured - 1 is annually.
    amount = principle * (((1 + ((rate/100.0)/n)) ** (n*time)))
    return(amount)
    
def interestToDate(loanSchedule, currentMark):
    interestPaid = 0
    for i in range(1,currentMark*12):
        values = installmentToDict(loanSchedule.schedule(currentMark))
        interestPaid = interestPaid + values['interest']
    return interestPaid

def principleToDate(loanSchedule, currentMark):
    principalPaid = 0
    for i in range(1,currentMark*12):
        values = installmentToDict(loanSchedule.schedule(currentMark))
        principalPaid = principalPaid + values['principal']
    return principalPaid

def format(x):
        return "${:.1f}K".format(x/1000)
    
def getDisplayValues(markValues):
    displayItems = ["purchasePrice", "deposit", "balance", "year", "valueAtMark", "capitalGains", "personalGains","gainsOnDeposit", "weeklyMortgagePayment"]
    return  {k: markValues[k] for k in displayItems}

    
## for changing the slider variables
def on_change(v):
    x = v['new'] 

    
    

In [2]:
# set some defaults

## assumed rate of capital gains, default to the Auckland average of 5%
defaultCapitalGainsRate=5.2
## the number of years. 
defaultMark= 5
## interest rate, expressed as a value between 0 and 1
defaultInterestRate= 4
## the number of years the loan would run for if I didn't cash out. 
defaultTerm = 30
## total value of the hypothetical property
defaultTotalValue = 650000
## value of deposit from Sam
defaultDepositValue = 70000

def getMarkValues(capitalGainsRate, mark, interestRate, term, totalValue, depositValue):
    ## sum borrowed from the bank
    loanValue = totalValue - depositValue

    ## minimum house value at the mark, assuming capital gains rate stays at at least the default
    finalValue = compound_gains(loanValue, capitalGainsRate, mark)

    ## loan computation
    loan = Loan(principal=loanValue, interest=interestRate, term = term)

    valuesAtMark = installmentToDict(loan.schedule(mark*12))
    
    valuesAtMark['purchasePrice'] = totalValue
    valuesAtMark['deposit'] = depositValue
    valuesAtMark['year'] = mark
    valuesAtMark['interestToDate'] = interestToDate(loan, mark)
    valuesAtMark['interestToDate'] = valuesAtMark.pop('total_interest')
    valuesAtMark['principleToDate'] = principleToDate(loan, mark)

    # capital gains
    valuesAtMark['valueAtMark'] = compound_gains(totalValue, capitalGainsRate, mark)
    valuesAtMark['capitalGains'] = valuesAtMark['valueAtMark'] - totalValue
    valuesAtMark['personalGains'] = valuesAtMark['capitalGains']*(1-(depositValue/totalValue)) + loanValue-float(valuesAtMark['balance'])
    valuesAtMark['gainsOnDeposit'] = valuesAtMark['capitalGains']*(depositValue/totalValue)

    valuesAtMark['interestOnDeposit'] = (depositValue * interestRate) * mark

    valuesAtMark['weeklyMortgagePayment'] = float(loan.monthly_payment) * 12/52
    valuesAtMark['weeklyPayment'] = valuesAtMark['weeklyMortgagePayment'] + (depositValue * interestRate)/52
    
    for key, value in valuesAtMark.items():
        if key != "year" :
            valuesAtMark[key] = round(float(valuesAtMark[key]), 2)

    #markValues = pd.DataFrame(valuesAtMark.items(), columns =["field", "value"]) 
    #markValues['value'] = markValues['value'].apply('${:,.2f}'.format)
    #markValues.iloc[2,1] = valuesAtMark['number']

    return valuesAtMark



## Kiwibuild, new build

I currently fulfill the eligibility criteria for Kiwibuild - I don't currently own a home, I live in New Zealand and I earn less than 120K. Notable caveats are that it requires any loan/gift for a deposit to be declared a gift and not be expected to be repaid, and you have to live there for a year if you get a 1 bedroom, or 3 years if you get a 2 or more bedroom place. 

There are currently builds available in Auckland, though they are entertainingly difficult to get details on. For example, of the ones out west currently listed on https://www.kiwibuild.govt.nz/available-homes/, 
- Titoki Street in Te Atatu, can find no information.
- Aroha in Avondale, sold out.  
- Plus Pacific Tower in Henderson, there is a working contact email (not that they have replied yet), but no information regarding what is available or not. 

Have not yet investigated the southern ones, but will get to it. A quick glance leaves me not overly sanguine with regards to the availability of information. 



I've calculated a default kiwibuild status at 5 years. And the controls below should enable comparisons to be made if needed. 

In [55]:
pd.options.display.float_format = "{:,.2f}".format
kiwibuildMark = getMarkValues(5, 5, 0.04, 30, 550000, 32500)
displayValues = getDisplayValues(kiwibuildMark)
displayDf = pd.DataFrame(displayValues.items(), columns =["field", "value"])
displayDf.style.apply(lambda x: round(2))
display(HTML(displayDf.to_html(index=False, header=False)))


purchasePrice,"550,000.00"
deposit,"32,500.00"
balance,"468,065.88"
year,5.00
valueAtMark,"701,954.86"
capitalGains,"151,954.86"
personalGains,"192,409.83"
gainsOnDeposit,"8,979.15"
weeklyMortgagePayment,570.14


For kiwibuild, I am defaulting to a property price of around 650K and a 5\% deposit of 32500. At the five year mark this gives me {{kiwibuildMark["personalGains"]}}. This is assuming that the rate of capital gains in Auckland doesn't drop below the average for the last few years of 5.2%, and that interest rates don't rise about 4\%. It would also mean that the weekly mortgage payment is {{kiwibuildMark["weeklyMortgagePayment"]}}, which is easily do-able with a flatmate. The interest on the deposit would be {{kiwibuildMark["interestOnDeposit"]}}, which could either be paid by bringing the weekly payment up to {{kiwibuildMark["weeklyPayment"]}} or taken off the gains at the end of the arrangement. 

The total cost has to drop well under five hundred before the weekly payment is do-able for a single bedroom. 

The other thing of note in this is that I have split the capital gains by figuring out what proportion of the total value the deposit is, and multiplying capital gains by that ratio. We should think/talk about that, as it means the gains on the deposit are smaller.

Entertainingly, it pays off in the long run to buy something more expensive, as whilst less is paid off due to more going towards interest rather than principle, the capital gains outweighs that. 


In [65]:
style = {'description_width': '150px', "description_position":"left"}
buttonstyle = {'description_width': '150px', "object_position":"right"}
form_item_layout = Layout(
    width='375px'
)

totalValue=widgets.IntText(value=defaultTotalValue, step=1000, description = "Sale Price", style=style)
depositValue=widgets.IntText(value=defaultDepositValue, step=1000, description = "Deposit", style=style)
mark=widgets.IntText(value=defaultMark, description = "Status at year (x): ", style=style)
interestSlider = widgets.FloatSlider(value=defaultInterestRate, min=0, max=10, description = "Interest rate: ", style=style, layout=form_item_layout)
interestSlider.observe(on_change, names='value')
capitalGainsSlider = widgets.FloatSlider(value=defaultCapitalGainsRate, min=0, max=10, description = "Capital gains rate: ", style=style, layout=form_item_layout)
capitalGainsSlider.observe(on_change, names='value')
ermSlider = widgets.IntSlider(value=defaultTerm, min=0, max=30, description = "Loan term: ", style=style, layout=form_item_layout)
termSlider.observe(on_change, names='value')

#interestRate = interestSlider.value/100
#capitalGainsRate = capitalGainsSlider.value/100

button = widgets.Button(description="Calcuate", style = buttonstyle)
button.on_click(on_button_clicked)
output = widgets.Output()
#display(button, output)

box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='flex-end',
                width='75%')


thing = AppLayout(header=None,
          left_sidebar=VBox([totalValue,depositValue,mark,interestSlider,capitalGainsSlider,termSlider, HBox([button], layout=box_layout)] ),
          center=None,
          right_sidebar=output,
          footer=None)

def on_button_clicked(b):
    output.clear_output()
    with output:
        markValues = getMarkValues(capitalGainsSlider.value, mark.value, interestSlider.value/100, termSlider.value, totalValue.value, depositValue.value)
        displayValues = getDisplayValues(markValues)
        displayDf = pd.DataFrame(displayValues.items(), columns =["field", "value"])
        displayDf['value'] = displayDf['value'].round(2)
        displayDf.style.apply(lambda x: round(2))
        display(HTML(displayDf.to_html(index=False, header=False)))



thing

AppLayout(children=(VBox(children=(IntText(value=650000, description='Sale Price', step=1000, style=Descriptio…

## New build in Auckland

This doesn't really change any of the calculations above. The only thing it does do is up the required deposit. There is still a ceiling of whatever it is that the bank is willing to lend. If it's not Kiwibuild then the only viable option for a new build would be a small townhouse or apartment. 

The Kiwibuild developments tend to offer small floor areas - Ockham's next Kiwibuild project for example is offering one bedroom apartments from 42 to 45 sqm. 

## Existing build in Auckland

The options for purchasing an existing build in Auckland are limited. With a hard ceiling from the bank, these are likely to be either a small brick and tile or a townhouse. Neither of these are objectionable. Even with these however, there are limits. If one were to luck into one of these then it would be Avondale and west, or Otahahu and south, including Mangere, and Mangere Bridge. 

An alternative option, is the purchase of a do-up. There are a small number of properties in need of attention that may go for an affordable sum. Obviously major structural problems would rule a property out, but the option exists for additional value to be added by virtue of free labor (mine).

## Out of Auckland

I have thought about this. I really have. And the answer is no. 


## Leasehold


Leasehold is a little different. The upfront sum is significantly less. There is a significant cost involved in the ground rent though. The upfront sum could be anywhere from 150 to 250k. On the face of it, it would be very easy to pay this off in a few years. Ground rent can easily add 300 pw to the costs though, making repayments smaller. Things to be wary about - when is the ground rent due for review, CVu had a thing last year where an extra 10K/year was added to the ground rent. A review just gone might make this an attractive option for a couple of years, or a review several years away, giving time to buy, recoup and sell with a signiicant period left before the next review - will be harder to sell if the next review is close. 

In [5]:



propertyValueInput=widgets.IntText(value=200, description= "Total Value(k): ", style = style)
display(propertyValueInput)

depositInput=widgets.IntText(value=40, description= "Deposit(k): " , style = style)
display(depositInput)

groundRentInput=widgets.IntText(value=12, description= "Ground rent(k): ", style = style)
display(groundRentInput)

x=3
interestSlider = widgets.FloatSlider(value=3, min=0, max=10, description="Interest rate(%)" , readout_format='.2f', style = style)
interestSlider.observe(on_change, names='value')
display(interestSlider)


termInput=widgets.IntText(value=10, description= "Term(y): ", style = style)
display(termInput)


IntText(value=200, description='Total Value(k): ', style=DescriptionStyle(description_width='150px'))

IntText(value=40, description='Deposit(k): ', style=DescriptionStyle(description_width='150px'))

IntText(value=12, description='Ground rent(k): ', style=DescriptionStyle(description_width='150px'))

FloatSlider(value=3.0, description='Interest rate(%)', max=10.0, style=SliderStyle(description_width='150px'))

IntText(value=10, description='Term(y): ', style=DescriptionStyle(description_width='150px'))

If we add a proviso of limiting the rent to around $700 - flexible, but roughly 500(-100) for me, 200(+/-100) for a flatmate, then paying around with the settings, we can see that with ~300pw added for ground rent, then we have to have a term of ~10 years to make the rent achievable. 

In [6]:
interestRate= interestSlider.value/100
totalValue = propertyValueInput.value * 1000
depositValue=depositInput.value * 100
term = termInput.value
loan = Loan(principal=totalValue-depositValue, interest=interestRate, term=term)


fiveYearMark = installmentToDict(loan.schedule(60))

weeklyMortgage = (fiveYearMark['payment']*12)/52
weeklyGroundRent = groundRentInput.value*1000/52
weeklyMortgage = float(weeklyMortgage)
weeklyPayment = weeklyMortgage + weeklyGroundRent

print("weekly mortgage payment:")
print("${:,.2f}".format(weeklyMortgage))
print("weekly payment including ground rent/opex:")
print("${:,.2f}".format(weeklyPayment))


weekly mortgage payment:
$436.75
weekly payment including ground rent/opex:
$667.52


So a shorter term, with more goes towards the capital. If we look at this at the five year mark then what remains is:

In [7]:
fiveYearMark = installmentToDict(loan.schedule(60))
print ("${:,.2f}".format(totalValue-depositValue-fiveYearMark["balance"]))



$90,672.87


Which might be fine if there were capital gains, but the general advice appears to be that the capital gains accrue to the land, and not the built structures. So total gains on this appear to be ~ 80K over five years. 

Most of these options have a cost - namely, the interest. This option has the added cost of 

In [8]:
groundRentPaid = weeklyGroundRent * 52 * term
print ("${:,.2f}".format(groundRentPaid))

$120,000.00


has been paid to get to this point, a combination of ground rent and opex in a roughly 2/3-1/3 split. 
As an aside, if one was okay with being a landlord, and could afford to buy one of these outright, it might possibly be an okay thing to contemplate, as the only cost is that of your ground rent/opex. 

## Commercial

Commercial properties require a minimum of 40% deposit, which makes them difficult to obtain finance for. Additionally, interest rates are higher, and loan terms shorter. 

Currently, the BNZ appears to be the only viable option as it is possible to get a 25 year term loan, as opposed to the standard 15 that other banks appear to have as their limit. 


Interest rates for the BNZ vary, with a base of 2.2% and an industry specific rate that can vary between 1.8 and 4. 

In [9]:

propertyValue=widgets.IntText(value=650, description= "Total Value: ")

display(propertyValue)

depositValue=widgets.IntText(value=120, description= "Deposit: ")

display(depositValue)


x = 5
slider = widgets.IntSlider(value=4, min=0, max=10)
# change the slider value
def on_change(v):
    x = v['new'] 
slider.observe(on_change, names='value')

display(slider)

IntText(value=650, description='Total Value: ')

IntText(value=120, description='Deposit: ')

IntSlider(value=4, max=10)

In [10]:


interestRate= slider.value/100

totalValue = propertyValue.value * 1000
depositValue=depositValue.value * 100

loan = Loan(principal=totalValue-depositValue, interest=interestRate, term=25)



At the 5 year mark, the capital that I would hold would be:
Value of property - sams deposit - sams share of capital gains - whatever is left to the bank. 

In [11]:


fiveYearMark = installmentToDict(loan.schedule(60))

for key, value in fiveYearMark.items():
    if key=="number":
        print(key, " : ", value)
    else:
        print(key, " : ", "${:,.2f}".format(value))


balance  :  $555,727.45
interest  :  $1,857.46
number  :  60
payment  :  $3,367.60
principal  :  $1,510.14
total_interest  :  $119,783.39


So my capital becomes is what remains when the deposit and what the bank still owns is removed form the total value:

In [12]:

print ("${:,.2f}".format(totalValue-depositValue-fiveYearMark["balance"]))

$82,272.55


This is without accounting for capital gains. Auckland has been at ~5% for some time, so I'm currently seeting capital gains at 4%, which leaves a total value of:

In [13]:
gainsRate=5
mark=5
  
finalValue = compound_interest(totalValue, gainsRate, mark)
print("${:,.2f}".format(finalValue))



NameError: name 'compound_interest' is not defined

And a final total capital for myself of:

In [14]:
personalGains = float((finalValue-totalValue)/2)
fiveYearMark["balance"] = float(fiveYearMark["balance"])
      
print ("${:,.2f}".format(totalValue + personalGains - depositValue - fiveYearMark["balance"]))

NameError: name 'finalValue' is not defined

Then the question remains, how much has this cost me? The two sums I am interested in here are how much have I spent, and how much would I have spent on rent/saved otherwise. 
Costs at this point will have been the interest on the loan, plus the interest on the deposit. 
Interest on the loan to this point:

In [15]:
interestPaid = 0
for i in range(1, 5*12):
    interestPaid = interestPaid + loan.schedule(i).interest
    
print("${:,.2f}".format(interestPaid))

$117,925.94


whilst what I would have paid in rent (300pw) would have been:

In [16]:
rent = 200 * 52 * 5
print(rent)

52000


and I would have been able to save maybe ...

In [17]:
savings = 350 * 52 * 5
print(savings)

91000
